# Multiclass classification



## Learning objectives



- Understand how classification can be implemented when there are more than 2 classes
- Implement a multiclass classifier from scratch



## Binary vs Multiclass



In binary classification the output must be either `True` or `False` as we already know.

Either the example falls into this class, or it doesn't. We have seen that we can represent this by our model having a single output node whose value is forced between 0 and 1, and as such represents probability that the example belongs to the positive class.



## Multiclass


<p align=center><img width=1000 src=images/binary-class.jpg></p>

In the case where we have two nodes to represent true and false, we can think about it as having trained two separate models.

Treating `True` and `False` as separate classes with separate output nodes shows us how we can extend this idea to do multiclass classification; 

> <font size=+2>We simply add more nodes and ensure that their values are positive and sum to one.</font>

__Each node is a single `logit` and all of them combined are later passed to `softmax`__

<p align=center><img width=1000 src=images/multiclass.jpg></p>



## Multiclass vs Multilabel



In this course we will not talk about __multilabel__ case, but:

> In multilabel problem, each label can exist simultaneously instead of exclusively like in multiclass

This might be a single vector where we have `cat` and `dog` on a picture but not a turtle:

$$
[1, 0, 1]
$$

> In multiclass, __there is always a single `1` label__, not less, not more

## Logits



Here we will also outputs logits, in case of multiclass the only change is __it will be a vector of values__. Each value in the output vector corresponds to certain class.

Let's say we would like to classify our input image into one of threes classes: `{dog=0, cat=1, turtle=2}`. Output of our model might look like this:

$$
    [-5, -3, 2]
$$

This would be a prediction of class `turtle` as it's value is highest.
When we want to get a label from this operation we use [`argmax`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html):

> <font size=+2> `argmax` returns __index__ of array entry with __the highest value__ </font>

As before, we can transform them into probabilities using...



## Softmax


> <font size=+2>The **softmax function** exponentiates each value in a vector to make it positive and then divides each of them by their sum to normalise them (make them sum to 1). </font>

This ensures that the vector then can be interpreted as a probability distribution.

<p align=center><img width=1000 src=images/softmax.jpg></p>

So, for example, we can replace each variable with some values

<p align=center><img width=1000 src=images/softmax_example.jpg></p>



## Differentiating the softmax



- Softmax derivative is different based on the index of element with respect to which we take derivative. 
- If it is the same as the index of element we applied softmax, the derivative is the equation at the bottom
- Otherwise the one above it. 

<p align=center><img width=1000 src=images/softmax_deriv.jpg></p>



### Properties of softmax



- increasing the value of any entry decreases the value of all of the others, because the whole vector must always sum to one. 
- an increase in one input element increases it's corresponding output element exponentially whilst pushing others down, 
- this means that __it is very easy for the one largest output element to become dominant__
- because of that `softmax` is overconfident and there are ways to combat this like `label smoothing`



### What does the name "softmax" mean?



- as explained above, usually one input is near `1`, and all others close to `0`. That is, similar to the `argmax` operation mentioned previously but "soft" as it can be differentiated.
- `argmax` changes abruptly, small difference between two values make it either `0` or `1`. Softmax on the other hand changes gradually when the maximum changes



## Exercise

Let's implement our own softmax function.

It should take `x` and divide by `sum` across `axis=1` (as we are normalizing along features):

In [1]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    exponentials = np.exp(x)
    return exponentials / exponentials.sum(axis=1).reshape(-1, 1)

## Stable softmax

As seen in `sigmoid` case this version also suffers from numerical instability, check out below:

In [2]:
softmax(np.array([[1000, 9, 8], [11, 12, 15]]))

<ipython-input-1-1a709be775cf>:5: RuntimeWarning: overflow encountered in exp
  exponentials = np.exp(x)
<ipython-input-1-1a709be775cf>:6: RuntimeWarning: invalid value encountered in true_divide
  return exponentials / exponentials.sum(axis=1).reshape(-1, 1)


array([[       nan, 0.        , 0.        ],
       [0.01714783, 0.04661262, 0.93623955]])

This time the result is even worse as we get `np.nan` due to overflow. Solution to the problem is to subtract maximum value from each row.

### Subtracting, what?

As `softmax` works along the horizontal axis (`1`) and all values sum to `1` the only thing that matters with the numbers in certain row is their absolute distance. 

This means we can divide __any value__ from them and still get the same results:

In [3]:
original = np.array([5, -2, 0]).reshape(1, -1)
subtracted = original - 6
softmax(original), softmax(subtracted)

(array([[9.92408247e-01, 9.04959183e-04, 6.68679417e-03]]),
 array([[9.92408247e-01, 9.04959183e-04, 6.68679417e-03]]))

### What to subtract?

There is no way to know what is the right `const` value to remove from each row. What if we have `1000` or `1_000_000`? 

Fortunately, we can find maximum in whole batch of data and simply subtract that.

## Exercise

Implement `softmax` function again, this time a stable version:
- subtract `np.max` from `logits` across `1` axis again
- return exponential values calculated this way like previously

Here is our `stable softmax`:

In [4]:
def softmax(logits):
    exps = np.exp(logits - np.max(logits, axis=1).reshape(-1, 1))
    return exps / np.sum(exps, axis=1).reshape(-1, 1)

In [5]:
softmax(np.array([[1000, 9, 8], [11, 12, 15]]))

array([[1.        , 0.        , 0.        ],
       [0.01714783, 0.04661262, 0.93623955]])

## One hot encoding

Our targets can be encoded in multiple ways. Usually, we would simply pass class numbers like this (for `5` samples):

$$
[0, 3, 1, 1, 4]
$$

We could also do that using one-hot encoding:

$$
\begin{align}
&[1, 0, 0, 0, 0]\\
&[0, 0, 0, 1, 0]\\
&[0, 1, 0, 0, 0]\\
&[0, 1, 0, 0, 0]\\
&[0, 0, 0, 0, 1]\\
\end{align}
$$

As most of the data works with the first option, we will now code how to transform `labels` into one-hot-encoding and vice versa:

In [6]:
def to_one_hot(labels, max_labels: int = None):
    if max_labels is None:
        max_labels = np.max(labels) + 1
    return np.eye(max_labels)[labels]


def to_labels(one_hot):
    return np.argmax(one_hot, axis=-1)


data = np.array([0, 1, 0, 3, 5])
to_one_hot(data)

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

One Hot Encoding can also be applied in the inputs, but be careful with using the whole output directly into the model. What problem can you think about this? 

One hint: look at this sklearn library called preprocessing:

In [3]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first')

Notice that one of the arguments we can pass to it is `drop`. Why do you think we want to drop a column from the One Hot Encoder output?

<details>
  <summary>Click here to see the answer</summary>

  When you perform One-Hot encoding, all the columns in the output sum always one. This is because only one column is 1, the rest are 0. Now, the problem with that is that these columns are correlated. 
  
  The problem with correlation in machine learning models is that it can lead to numerically unstable solutions. Check [this page](https://en.wikipedia.org/wiki/Multicollinearity#Consequences) to know more about it
  
  Therefore, when you perform One-Hot encoding, you should drop one column from the output.

</details>

## The cross entropy loss function

An appropriate loss function to use for multiclass classification is the __cross entropy loss function__.

- It is a __generalization of binary cross entropy loss__ so it will work in binary case as well
- BCE (binary cross-entropy) is faster and more stable for binary case so __it should be created separately__ and __used separately__.

Like BCE loss, cross entropy uses the same term: __the negative natural log of the output probability__ to penalise outputs exponentially as they stray from the ground truth.

> We don't need to simultaneously push down the incorrect class probabilities and push up the correct class probabilities.

So if we focus on increasing the correct class likelihood element, then we will implicitly be decreasing the incorrect class likelihood elements.

<p align=center><img width=1000 src=images/cross_entropy_loss.jpg></p>

## Where to use simple linear models?

We have seen how to create and use `linear models` for regression and classification.
Soon we will meet more powerful models but here is the rough summary of when one should use it in real life:
- as a baseline - gives us an overview and "starting point" for improvement
- when we want easily explaianble model. Each weight shows the impact of a factor onto our target
- when we have a lot of features (even more than data point) and we do not want to overfit on data

With experience and more models it will become increasingly clear where we should use each.

## Multiclass classification in sklearn

The same way you can implement binary classification in sklearn, you can also implement multiclass classification. You can also use logistic regression for this purpose, however, in this case, instead of loading a dataset with two possible outputs, we are loading the `iris` dataset, which has three possible outputs.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X, y = load_iris(return_X_y=True, as_frame=True)
display(X)
display(y)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int64

You can see that the dataset has 150 samples, 4 features, and 3 different classes, each one corresponding to a different species of flower.

Let's split the dataset into a training set and a test set, and train the logistic regression. One argument that you can pass to the LogisticRegression class is multi_class. By default, it is set to `auto`, which selects `ovr` for binary classification and `multinomial` for multiclass classification. `ovr` means One-vs-Rest, which calculates the probabilities for each pair of classes, like in a binary classification problem. `multinomial` calculates the probability distribution for all the classes, however, it doesn't mean it's always more suitable for multiclass classification.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
log_reg = LogisticRegression(multi_class='multinomial', solver='newton-cg')
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
print(accuracy_score(y_test, y_pred))

1.0


Not bad! The iris dataset is a toy dataset, so it's actually not very difficult to train a good model on it. Be suspicious if you get a score of 1.0!

## Summary

- multiclass classification is multiple linear regression stacked together
- multiclass classification requires a different loss function (cross entropy)
- we can work directly on logits to take predictions by using `argmax`
- softmax is a differentiable function that turns a vector of real numbers into a probability distribution